In [1]:
import dill
dill.load_session('ref_reports.db')

In [1]:
import pandas as pd 
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [2]:
path = 'C:/Users/isaac/Dropbox/Apps/ShareLaTeX/information_lawyer_quality'

# Read data 
data = pd.read_csv(path + '/_aux/hd_data.csv') 
#data = data.sample(frac=0.1, replace=True, random_state=1)
print(data.shape)
data.columns

(4998, 77)


Index(['reclutamiento', 'giro_empresa_d2', 'giro_empresa_d3',
       'giro_empresa_d4', 'giro_empresa_d5', 'giro_empresa_d6',
       'giro_empresa_d7', 'giro_empresa_d8', 'giro_empresa_d9',
       'giro_empresa_d10', 'giro_empresa_d11', 'giro_empresa_d12',
       'giro_empresa_d13', 'giro_empresa_d14', 'giro_empresa_d15',
       'giro_empresa_d16', 'giro_empresa_d17', 'giro_empresa_d18',
       'giro_empresa_d19', 'giro_empresa_d20', 'giro_empresa_d21',
       'giro_empresa_d22', 'giro_empresa_d23', 'giro_empresa_d24',
       'trabajador_base', 'gen', 'tipo_jornada_d2', 'tipo_jornada_d3',
       'tipo_jornada_d4', 'reinst', 'indem', 'sal_caidos', 'prima_antig',
       'prima_vac', 'hextra', 'rec20', 'prima_dom', 'desc_sem', 'desc_ob',
       'sarimssinf', 'utilidades', 'monto_recsueldo', 'nulidad', 'vac', 'ag',
       'codem', 'horas_extra', 'antiguedad', 'salario_diario', 'horas_sem',
       'c_antiguedad', 'c_indem', 'c_prima_antig', 'c_rec20', 'c_ag', 'c_vac',
       'c_hextra', 'c_

In [3]:
data.describe()

,reclutamiento,giro_empresa_d2,giro_empresa_d3,giro_empresa_d4,giro_empresa_d5,giro_empresa_d6,giro_empresa_d7,giro_empresa_d8,giro_empresa_d9,giro_empresa_d10,...,min_vac,min_prima_vac,min_ley,c_sal_caidos,prop_hextra,modo_termino,liq_total,liq_total_tope,liq_total_pos,duracion
count,4997.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,4998.000000,...,4871.000000,4871.000000,4.998000e+03,4.123000e+03,4984.000000,4998.000000,4.998000e+03,4.998000e+03,4998.000000,4998.000000
mean,0.192516,0.000400,0.000400,0.002601,0.034014,0.003601,0.007003,0.052621,0.232093,0.022809,...,1938.003057,484.500675,5.269175e+04,2.294548e+05,0.369944,1.605842,2.854584e+04,2.761328e+04,0.653661,1.021339
std,0.394315,0.020002,0.020002,0.050939,0.181282,0.059910,0.083398,0.223298,0.422210,0.149310,...,6396.703136,1599.175766,1.089357e+05,5.586934e+05,0.314853,0.933609,1.400880e+05,1.275581e+05,0.475850,0.958674
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.990000,0.250000,0.000000e+00,1.394670e+03,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.005479
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,352.628555,88.155888,2.044597e+04,7.340045e+04,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.331507
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,848.219970,212.050000,3.172866e+04,1.178750e+05,0.394096,1.000000,8.000000e+03,8.000000e+03,1.000000,0.697260
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1867.498600,466.874645,5.436342e+04,2.186237e+05,0.641515,2.000000,2.073375e+04,2.073375e+04,1.000000,1.400000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,213154.980000,53288.746000,4.449881e+06,2.005620e+07,0.996460,4.000000,6.700000e+06,5.600418e+06,1.000000,4.960000


In [4]:
X = data[['reclutamiento',
'trabajador_base',
'gen',
'tipo_jornada_d2',
'tipo_jornada_d3',
'tipo_jornada_d4',
'reinst',
'indem',
'sal_caidos',
'prima_antig',
'prima_vac',
'hextra',
'rec20',
'prima_dom',
'desc_sem',
'desc_ob',
'sarimssinf',
'utilidades',
'monto_recsueldo',
'nulidad',
'vac',
'ag',
'codem',
'horas_extra',
'antiguedad',
'salario_diario',
'horas_sem',
'c_antiguedad',
'c_indem',
'c_prima_antig',
'c_rec20',
'c_ag',
'c_vac',
'c_hextra',
'c_prima_vac',
'c_prima_dom',
'c_desc_sem',
'c_desc_ob',
'c_utilidades',
'c_recsueldo',
'c_total',
'min_indem',
'min_prima_antig',
'min_ag',
'min_vac',
'min_prima_vac',
'min_ley',
'c_sal_caidos',
'prop_hextra'
]]


sett = data['modo_termino']==1
sett.astype('int64')
cr_win = (data['modo_termino']==3) & (data['liq_total_pos']==1)
cr_win.astype('int64')
cr_los = (data['modo_termino']==3) & (data['liq_total_pos']==0)
cr_los.astype('int64')

X_3 = X[data['modo_termino']==3] 
rslt_df3 = data.loc[data['modo_termino']==3]
cr_win_ph2 = rslt_df3['liq_total_pos']==1
cr_win_ph2.astype('int64')


X_1 = X[data['modo_termino']==1]
rslt_df1 = data.loc[data['modo_termino']==1]
tot_comp_1l = np.log(rslt_df1['liq_total_tope']+1)

tot_comp_3l = np.log(rslt_df3['liq_total_tope']+1)

tot_comp_3 = rslt_df3['liq_total_tope']


In [5]:
numeric_featuresX = ['antiguedad',
'salario_diario',
'horas_sem',
'c_antiguedad',
'c_indem',
'c_prima_antig',
'c_rec20',
'c_ag',
'c_vac',
'c_hextra',
'c_prima_vac',
'c_prima_dom',
'c_desc_sem',
'c_desc_ob',
'c_utilidades',
'c_recsueldo',
'c_total',
'min_indem',
'min_prima_antig',
'min_ag',
'min_vac',
'min_prima_vac',
'min_ley',
'c_sal_caidos',
'prop_hextra']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())])




binary_featuresX = ['reclutamiento',
'trabajador_base',
'gen',
'tipo_jornada_d2',
'tipo_jornada_d3',
'tipo_jornada_d4',
'reinst',
'indem',
'sal_caidos',
'prima_antig',
'prima_vac',
'hextra',
'rec20',
'prima_dom',
'desc_sem',
'desc_ob',
'sarimssinf',
'utilidades',
'monto_recsueldo',
'nulidad',
'vac',
'ag',
'codem',
'horas_extra']

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=1, include_bias=False))])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresX),
        ('bin', binary_transformer, binary_featuresX)])


In [6]:
from sklearn.model_selection import train_test_split

[X_train, X_test, sett_train, sett_test, cr_los_train, cr_los_test,
 cr_win_train, cr_win_test] = train_test_split(X, sett, cr_los, cr_win, test_size=0.2, random_state=0)


[X_1_train, X_1_test, tot_comp_1l_train, tot_comp_1l_test] = train_test_split( X_1, tot_comp_1l, 
                                                                  test_size=0.2, random_state=0)


[X_3_train, X_3_test, cr_win_ph2_train, cr_win_ph2_test,
tot_comp_3l_train, tot_comp_3l_test,tot_comp_3_train, tot_comp_3_test] = train_test_split( X_3, cr_win_ph2, 
                                                                                          tot_comp_3l, tot_comp_3, 
                                                                  test_size=0.2, random_state=0)

In [12]:
from sklearn.linear_model import LogisticRegression
# Hyperparameters - grid
# ==============================================================================
param_grid = {'model__penalty'  : ['l1', 'l2', 'elasticnet'],
              'model__C'     : [0.0001, 0.001, 0.01, 0.1, 1, 10, 20]
             }


logit_est = LogisticRegression(random_state = 0,
                              max_iter = 10000)

# Logit Model
LOGIT = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', logit_est)])


# Grid search by cross-validation
# ==============================================================================
grid = GridSearchCV(LOGIT,
        param_grid = param_grid,
        scoring    = 'f1_weighted',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

# Recover the best model
grid.fit(X_train, sett_train)
model_sett = grid.best_estimator_
pr_sett = model_sett.predict_proba(X_test)[:,1]


C:\Users\isaac\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.32145899        nan        nan 0.32184076        nan
        nan 0.32145899        nan        nan 0.32184076        nan
        nan 0.32184076        nan        nan 0.32184076        nan
        nan 0.32184076        nan]
  warnings.warn(
C:\Users\isaac\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the train scores are non-finite: [       nan 0.31984896        nan        nan 0.32074565        nan
        nan 0.32004558        nan        nan 0.32074565        nan
        nan 0.32074565        nan        nan 0.32074565        nan
        nan 0.32074565        nan]
  warnings.warn(


In [29]:
pr_sett = model_sett.predict_proba(X_train)[:,1]

In [30]:
    cor = np.corrcoef(sett_train, pr_sett)
    cor
    

array([[1.        , 0.03253379],
       [0.03253379, 1.        ]])

In [33]:
    fpr, tpr, thresholds = metrics.roc_curve(sett_train, pr_sett, pos_label=1)
    metrics.auc(fpr, tpr)
    

0.521449991889526

In [34]:
accuracy_score(sett_train, np.round(pr_sett))

0.39269634817408705

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__n_estimators': [500],
              'model__max_features': [5, 7, 9]
             }



RFC = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', RandomForestClassifier(random_state=123))])


# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = RFC,
        param_grid = param_grid,
        scoring    = 'f1_weighted',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

# Recover the best model
grid.fit(X_train, cr_win_train)
model_cr_win = grid.best_estimator_
pr_cr_win = model_cr_win.predict_proba(X_test)[:,1]

grid.fit(X_train, cr_los_train)
model_cr_los = grid.best_estimator_
pr_cr_los = model_cr_los.predict_proba(X_test)[:,1]

grid.fit(X_3_train, cr_win_ph2_train)
model_cr_win_ph2 = grid.best_estimator_
pr_cr_win_ph2 = model_cr_win_ph2.predict_proba(X_3_test)[:,1]


In [ ]:
    cor = np.corrcoef(cr_win_test, pr_cr_win)
    cor

In [7]:
from sklearn.ensemble import RandomForestRegressor


# Random Forests
RFX = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', RandomForestRegressor(random_state=123,
                                                     n_estimators=500))])



# Recover the best model
model_tot_comp_1lrf = RFX.fit(X_1_train, tot_comp_1l_train)
pr_tot_comp_1lrf = model_tot_comp_1lrf.predict(X_1_test)

In [11]:
pr_tot_comp_1lrf = model_tot_comp_1lrf.predict(X_1_test)

In [12]:
cor = np.corrcoef(tot_comp_1l_test, pr_tot_comp_1lrf)
cor
    

array([[1.        , 0.52681711],
       [0.52681711, 1.        ]])

In [13]:
np.sqrt(mean_squared_error(tot_comp_1l_test, pr_tot_comp_1lrf))

1.1779611596014585

In [14]:
explained_variance_score(tot_comp_1l_test, pr_tot_comp_1lrf)

0.2757608114941553

In [7]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

LRX = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', LinearRegression())])


model_tot_comp_3l = LRX.fit(X_3_train, tot_comp_3l_train)
pr_tot_comp_3l = model_tot_comp_3l.predict(X_3_train)

model_tot_comp_1l = LRX.fit(X_1_train, tot_comp_1l_train)
pr_tot_comp_1l = model_tot_comp_1l.predict(X_1_train)


model_tot_comp_3 = LRX.fit(X_3_train, tot_comp_3_train)
pr_tot_comp_3 = model_tot_comp_3.predict(X_3_train)



